# Customer Churn Prediction

**Goal:** Predict customers likely to churn using historical usage and support logs. This notebook shows preprocessing, SMOTE for imbalance, Logistic Regression and XGBoost, and evaluation with ROC, precision-recall.


In [ ]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc, confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('customer_data.csv')
df.head()

In [ ]:
# Quick EDA
print('Shape:', df.shape)
print('Churn rate:', df.churn.mean())
df.describe()

In [ ]:
# Preprocessing
X = df[['tenure_months','monthly_usage','support_calls','is_premium','last_login_days']]
y = df['churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

In [ ]:
# Handle imbalance with SMOTE on training set
sm = SMOTE(random_state=42)
X_sm, y_sm = sm.fit_resample(X_train_s, y_train)
print('Resampled shape', X_sm.shape, y_sm.mean())

In [ ]:
# Logistic Regression baseline
lr = LogisticRegression(max_iter=1000)
lr.fit(X_sm, y_sm)
pred = lr.predict(X_test_s)
print(classification_report(y_test, pred))
print('ROC AUC:', roc_auc_score(y_test, lr.predict_proba(X_test_s)[:,1]))

In [ ]:
# XGBoost model
import xgboost as xgb
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', n_estimators=200)
xgb_clf.fit(X_sm, y_sm)
pred_x = xgb_clf.predict(X_test_s)
print(classification_report(y_test, pred_x))
print('ROC AUC:', roc_auc_score(y_test, xgb_clf.predict_proba(X_test_s)[:,1]))

In [ ]:
# Precision-Recall curve
probs = xgb_clf.predict_proba(X_test_s)[:,1]
precision, recall, _ = precision_recall_curve(y_test, probs)
pr_auc = auc(recall, precision)
print('PR AUC:', pr_auc)

## Save the model
You can save the scaler and model with joblib for later use in a web service.